In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#  Installation and Importing Libraries
!pip install fastapi nest-asyncio pyngrok uvicorn ngrok
from pyngrok import ngrok
!ngrok authtoken <your-ngrok-token>
import pandas as pd
import numpy as np
import pydantic
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import Ridge
from sklearn import linear_model
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, Body
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pandas import DataFrame, concat

In [ ]:
# Reading User File
u_cols =  ['user_id', 'firstName', 'lastName', 'phone']
users = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-Journese/u.user', sep='|', names=u_cols, encoding='utf-8')
n_users = users.shape[0]
# Create a mapping from user_id to an integer index
user_ids = users['user_id'].unique()
user_id_to_index = {user_id: index for index, user_id in enumerate(user_ids)}

In [ ]:
# Reading Ratings File
r_cols = ['user_id', 'place_id', 'rating']
ratings_base = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-Journese/u.data', sep='|', names=r_cols, encoding='utf-8')
ratings_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-Journese/u.data', sep='|', names=r_cols, encoding='utf-8')
rate_train = ratings_base.values
rate_test = ratings_test.values

In [ ]:
# Reading Items File
i_cols = ['place id', 'place title' , 'Nghỉ dưỡng', 'Phong cảnh', 'Lịch sử', 'Mua sắm', 'Ẩm thực', 'Văn hoá', 'Phượt', 'Biển', 'Núi',
          'Thiên nhiên', 'Sống ảo/Gen z', 'Nghệ thuật', 'Xanh', 'Phổ biến', 'Tham quan']
items = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-Journese/u.item', sep='|', names=i_cols, encoding='utf-8')
n_items = items.shape[0]
X0 = items.values
X_train_counts = X0[:, -15:]
print(X_train_counts)
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()

In [ ]:
# Function to Get Items Rated by User
def get_items_rated_by_user(rate_matrix, user_id):
    y = rate_matrix[:,0]
    # Use the mapping to get the index corresponding to the user_id
    ids = np.where(y == user_id)[0]
    item_ids = rate_matrix[ids, 1]  # treat place_id as string
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)


In [ ]:
# Training the Model
d = tfidf.shape[1]
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))
place_ids = items['place id'].unique()
place_id_to_index = {place_id: index for index, place_id in enumerate(place_ids)}
for user_id in user_ids:
    ids, scores = get_items_rated_by_user(rate_train, user_id)
    if len(ids) == 0:  # Skip users with no ratings
        continue
    print(f"user_id: {user_id}, ids: {ids}")
    clf = Ridge(alpha=0.01, fit_intercept  = True)
    try:
        indices = [place_id_to_index[place_id] for place_id in ids]
        Xhat = tfidf[indices, :]
    except IndexError:
        print(f"IndexError for user_id: {user_id}, ids: {ids}")
        break
    clf.fit(Xhat, scores)
    W[:, user_id_to_index[user_id]] = clf.coef_
    b[0, user_id_to_index[user_id]] = clf.intercept_


Yhat = tfidf.dot(W) + b


In [ ]:
#  Function to Recommend Places for User
def recommend_places_for_user(user_id, n_recommendations=10):
    if user_id not in user_id_to_index:
        print('Invalid user_id')
        return
    user_index = user_id_to_index[user_id]
    rated_item_ids, _ = get_items_rated_by_user(rate_train, user_id)
    predicted_ratings = Yhat[:, user_index]
    sorted_indices = np.argsort(predicted_ratings)[::-1]
    unrated_sorted_indices = [i for i in sorted_indices if i not in rated_item_ids]
    top_n_recommendations = unrated_sorted_indices[:n_recommendations]
    return top_n_recommendations

In [ ]:
# FastAPI Setup
app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)
@app.get("/recommendations/{user_id}")
async def get_recommendations(user_id: str):
    recommendations = recommend_places_for_user(user_id)
    place_titles = [{'id': items.iloc[place_id]['place id'], 'title': items.iloc[place_id]['place title']} for place_id in recommendations]
    return {"user_id": user_id, "recommendations": place_titles}


In [ ]:
# # API add place
# # Define a Pydantic model for Place data
class Place(BaseModel):
    id: str
    title: str
    tags: list[str]

@app.post("/places/")
async def create_or_update_place(place: Place = Body(...)):
    global items
    all_tags = ['Nghỉ dưỡng', 'Phong cảnh', 'Lịch sử', 'Mua sắm', 'Ẩm thực', 'Văn hoá', 'Phượt', 'Biển', 'Núi',
                'Thiên nhiên', 'Sống ảo/Gen z', 'Nghệ thuật', 'Xanh', 'Phổ biến', 'Tham quan']
    tag_values = [1 if tag in place.tags else 0 for tag in all_tags]
    place_data = [place.id, place.title] + tag_values
    if place.id in items['place id'].values:
        # Update the existing place
        items.loc[items['place id'] == place.id] = place_data
        message = "Place updated successfully!"
    else:
        # Append the new place data to the items DataFrame
        items = concat([items, DataFrame([place_data], columns=items.columns)], ignore_index=True)
        message = "Place created successfully!"

    # Update the item_id_to_index mapping
    items.to_csv('/content/drive/MyDrive/Colab Notebooks/ml-Journese/u.item', sep='|', header=False, index=False)

    # Update the tfidf array with the TF-IDF values of the new place
    global tfidf
    X0 = items.values
    X_train_counts = X0[:, -15:]
    transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
    tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()

    # Retrain the model
    global place_ids, place_id_to_index
    place_ids = items['place id'].unique()
    place_id_to_index = {place_id: index for index, place_id in enumerate(place_ids)}
    d = tfidf.shape[1]
    W = np.zeros((d, n_users))
    b = np.zeros((1, n_users))
    for user_id in user_ids:
        ids, scores = get_items_rated_by_user(rate_train, user_id)
        if len(ids) == 0:  # Skip users with no ratings
            continue
        print(f"user_id: {user_id}, ids: {ids}") 
        clf = Ridge(alpha=0.01, fit_intercept  = True)
        try:
            indices = [place_id_to_index[place_id] for place_id in ids]
            Xhat = tfidf[indices, :]
        except IndexError:
            print(f"IndexError for user_id: {user_id}, ids: {ids}")
            break
        clf.fit(Xhat, scores)
        W[:, user_id_to_index[user_id]] = clf.coef_
        b[0, user_id_to_index[user_id]] = clf.intercept_

    global Yhat
    Yhat = tfidf.dot(W) + b
    # Create a dictionary for the new place
    place_dict = {
        "id": place.id,
        "title": place.title,
        "tags": tag_values
    }

    return {"message": message, "place": place_dict}


In [ ]:
# API add or update rating
# Define a Pydantic model for Rating data
class Rating(BaseModel):
    user_id: str
    place_id: str
    rating: int

@app.post("/ratings/")
async def update_rating(rating: Rating = Body(...)):
    global ratings_base  # Declare ratings_base as global

    # Check if the rating already exists
    rating_exists = ((ratings_base['user_id'] == rating.user_id) & (ratings_base['place_id'] == rating.place_id)).any()

    if rating_exists:
        # If the rating exists, update it
        ratings_base.loc[(ratings_base['user_id'] == rating.user_id) & (ratings_base['place_id'] == rating.place_id), 'rating'] = rating.rating
    else:
        # If the rating doesn't exist, create it

        ratings_base = concat([ratings_base, DataFrame([rating.dict()])], ignore_index=True)

    # Write the updated DataFrame to the file
    ratings_base.to_csv('/content/drive/MyDrive/Colab Notebooks/ml-Journese/u.data', sep='|', header=False, index=False)
    # Update rate_train
    global rate_train
    rate_train = ratings_base.values
    # Retrain the model
    global place_ids, place_id_to_index
    place_ids = items['place id'].unique()
    place_id_to_index = {place_id: index for index, place_id in enumerate(place_ids)}
    d = tfidf.shape[1]
    W = np.zeros((d, n_users))
    b = np.zeros((1, n_users))
    for user_id in user_ids:
        ids, scores = get_items_rated_by_user(rate_train, user_id)
        if len(ids) == 0:  # Skip users with no ratings
            continue
        print(f"user_id: {user_id}, ids: {ids}")
        clf = Ridge(alpha=0.01, fit_intercept  = True)
        try:
            indices = [place_id_to_index[place_id] for place_id in ids]
            Xhat = tfidf[indices, :]
        except IndexError:
            print(f"IndexError for user_id: {user_id}, ids: {ids}")
            break
        clf.fit(Xhat, scores)
        W[:, user_id_to_index[user_id]] = clf.coef_
        b[0, user_id_to_index[user_id]] = clf.intercept_

    global Yhat
    Yhat = tfidf.dot(W) + b

    return {"message": "Rating updated successfully!", "rating": rating.dict()}



In [ ]:
class User(BaseModel):
    user_id: str
    firstName: str
    lastName: str
    phone: str
@app.post("/users/")
async def create_user(user: User = Body(...)):
    global users, user_ids, user_id_to_index, n_users
    users = concat([users, DataFrame([user.dict()])], ignore_index=True)
    users.to_csv('/content/drive/MyDrive/Colab Notebooks/ml-Journese/u.user', sep='|', header=False, index=False)
    # Update the user_id_to_index mapping
    user_ids = users['user_id'].unique()
    user_id_to_index = {user_id: index for index, user_id in enumerate(user_ids)}
    n_users = users.shape[0]

    # Retrain the model
    d = tfidf.shape[1]
    W = np.zeros((d, n_users))
    b = np.zeros((1, n_users))
    for user_id in user_ids:
        ids, scores = get_items_rated_by_user(rate_train, user_id)
        if len(ids) == 0:  # Skip users with no ratings
            continue
        clf = Ridge(alpha=0.01, fit_intercept  = True)
        try:
            Xhat = tfidf[ids, :]
        except IndexError:
            print(f"IndexError for user_id: {user_id}, ids: {ids}")
            break
        clf.fit(Xhat, scores)
        W[:, user_id_to_index[user_id]] = clf.coef_
        b[0, user_id_to_index[user_id]] = clf.intercept_

    # Update Yhat globally
    global Yhat
    Yhat = tfidf.dot(W) + b
    return {"message": "User created successfully!", "user": user}

In [ ]:
@app.get("/predictions/{user_id}")
async def get_predictions(user_id: str):
      # Check if user_id is in the users DataFrame
    if user_id not in users['user_id'].values:
        raise HTTPException(status_code=400, detail="Invalid user_id")

    # Get the rated item ids and scores
    rated_item_ids, rated_scores = get_items_rated_by_user(rate_train, user_id)

    # Get the predicted ratings for the user
    user_index = user_id_to_index[user_id]
    predicted_ratings = Yhat[:, user_index]

    # Prepare the response
    predictions = []
    for place_index, rating in enumerate(predicted_ratings):
        place_id = items.iloc[place_index]['place id']
        place_title = items.iloc[place_index]['place title']
        if place_id in rated_item_ids:
            true_rating_index = np.where(rated_item_ids == place_id)[0][0]
            true_rating = rated_scores[true_rating_index]
            predictions.append({"Place": place_title, "True Rating": true_rating})
        else:
            predictions.append({"Place": place_title, "Predicted Rating": rating})

    return {"user_id": user_id, "predictions": predictions}

In [ ]:
# Running the Server
ngrok_tunnel = ngrok.connect(8000, "http", hostname="exotic-filly-publicly.ngrok-free.app")
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)
